# Implementation KITTI-360 for KPRnet

**Authors:**
1. Aden Westmaas (4825373)
2. Badr Essabri (5099412)
3. Guido Dumont (5655366)

This Notebook contains the code to sample individual pointsclouds from the KITTI-360 dataset. These individual pointclouds are compatible with the source code of KPRNet.

*Remark:* The proposed methodology is computational expensive

In [1]:
import numpy as np
import open3d as o3d
from plyfile import PlyData, PlyElement
from tqdm.notebook import tqdm

## Original KITTI-360 data sample

The cells below visualizes one pointcloud sequences from the KITTI-360 dataset

In [2]:
original_pcl = o3d.io.read_point_cloud("kitti360_sequence.ply")
points = np.asarray(original_pcl.points)

In [3]:
# Get the pose of the car in world frame
def get_poses():
    poses = np.loadtxt("kitti360_poses.txt")
    frames = poses[:,0].astype(int)
    poses = np.reshape(poses[:,1:], (-1, 3, 4))

    add_rows = np.array([0, 0, 0, 1]).reshape((1, 1, 4))
    add_rows = np.repeat(add_rows, poses.shape[0], axis=0)

    poses = np.concatenate((poses, add_rows), axis=1)
    
    return frames, poses

frames, poses = get_poses()
print('Number of posed frames %d' % len(frames))

Number of posed frames 8185


In [4]:
# Sample pointcloud 80, translate to origin and visualize in world frame
index = 80

# Define origin
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10.0)
origin = origin.translate(poses[index][:3, 3:])
origin = origin.rotate(poses[index][:3, :3])
centre = origin.get_center()

# Translate points
translated_points = points.copy()
translated_points[:, 0] -= centre[0]
translated_points[:, 1] -= centre[1] + 3
translated_points[:, 2] -= centre[2] - 4.7
translated_points = np.dot(poses[index][:3, :3], translated_points.T).T

# Make pointcloud
translated_pcd = o3d.geometry.PointCloud()
translated_pcd.points = o3d.utility.Vector3dVector(translated_points)
translated_pcd.colors = original_pcl.colors

# Visualize
o3d.visualization.draw_geometries([o3d.geometry.TriangleMesh.create_coordinate_frame(size=10.0), translated_pcd])

## Resulting sampled KITTI-360 pointcloud

The cell below visualizes a sampled pointcloud. This sampled pointcloud is loaded in.

In [5]:
resulting_pcl = o3d.io.read_point_cloud("sampled_kitti360_pcl.pcd")

o3d.visualization.draw_geometries([o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0), resulting_pcl])

## Implementation algorithm

The cells below sample a single pointcloud from the sequence visualized above.

In [ ]:
# Transform to spherical_coordinates
def spherical_coordinates(data):
    output = np.zeros(data.shape)

    output[:, 0] = np.linalg.norm(data, axis=1) # r
    output[:, 1] = np.arccos(np.divide(data[:, 2], output[:, 0])) * (180 / np.pi) - 90 # Phi
    output[:, 2] = np.arctan2(data[:, 1], data[:, 0]) * (180 / np.pi) # Theta
    
    return output

# Sample individual pointclouds from the data
def sample(data):
    # Measurement range
    indeces = np.argwhere(data[:, 0] <= 120).ravel()
    sampled_points = data[indeces]

    # Define horizontale and vertical angles based on the specs of the LIDAR
    horizontal_angles = np.linspace(-180, 180, int(360 / 0.15))
    vertical_angles = np.linspace(-25, 2, 64)
    
    # Sample
    cloud_indeces = []
    for h in tqdm(horizontal_angles):
        for v in vertical_angles:
            close_points = np.sum(abs(sampled_points[:, 1:] - np.array([v, h])), axis=1)
            index = np.argmin(close_points)
            cloud_indeces.append(indeces[index])
            
    return cloud_indeces

spherical_points = spherical_coordinates(translated_points.copy())
cloud_indeces = sample(spherical_points)
cloud_indeces = np.random.choice(cloud_indeces, int(len(cloud_indeces)*0.1))

In [ ]:
# Visualize sampled pointclouds

sampled_pcl = o3d.geometry.PointCloud()
sampled_pcl.points = o3d.utility.Vector3dVector(translated_points[cloud_indeces])

colors = np.asarray(pcd.colors)
sampled_pcd.colors = o3d.utility.Vector3dVector(colors[cloud_indeces])

o3d.visualization.draw_geometries([o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0), sampled_pcd])

In [ ]:
# Save resulting pointcloud

# o3d.io.write_point_cloud("sampled_kitti360_pcl.pcd", sampled_pcd)